# SC-WEAT ANALYSIS

This notebook contains the SCWEAT score calculation for the earlier method - here, we use prebuilt GLoVe embeddings and vary the target set per topic (i.e. the target set for each topic consists of its respective top 20 words as obtained from BERTopic)

In [ ]:
from google.colab import drive
import os

  # gdrive_path='/content/gdrive/MyDrive/Bertopic/shared_work/'

  # # This will mount your google drive under 'MyDrive'
# drive.mount('/content/gdrive', force_remount=True)
# # In order to access the files in this notebook we have to navigate to the correct folder
# os.chdir(gdrive_path)
# dataset_path = ''
# # Check manually if all files are present
# print(sorted(os.listdir()))

# To run from the common drive:
dataset_path = '/content/drive/MyDrive/Praktikum - NLP Applications/Datasets/genius_and_wasabi/'
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# !pip install pandas bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 29.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 7.3 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039297 sha256=d2c069bf97aff4249d7c59d3f8f7c51dc5180473df56db356d1103377d11fd17
  Stored in dir

In [ ]:
!pip install pandas nltk

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import nltk
nltk.download('wordnet') # to use wordnet

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from nltk.corpus import wordnet as wn
# Find synsets for a word
synsets = wn.synsets('dog')
print(synsets)

[Synset('dog.n.01'), Synset('frump.n.01'), Synset('dog.n.03'), Synset('cad.n.01'), Synset('frank.n.02'), Synset('pawl.n.01'), Synset('andiron.n.01'), Synset('chase.v.01')]


In [ ]:
!pip install --upgrade tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.8 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0


In [ ]:
bertopic_folder = '/content/drive/MyDrive/Praktikum - NLP Applications/Models/bertopic_concatenated_chunks_stratified/'

df_path = bertopic_folder + 'topics_for_bias_analysis.csv'
aggregated_lyrics_path = bertopic_folder + 'aggregated_lyrics.csv'

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv(df_path)
aggregated_lyrics = pd.read_csv(aggregated_lyrics_path)

In [ ]:
aggregated_lyrics.head()

,Unnamed: 0,topic,gender,cleaned_lyrics,topic_label
0,0,-1,Female,"{entire song set to the melody of R. Kelly's ""...",love_im_dont
1,1,-1,Male,\n(*harmonizing*)\nAll my love is all I have\n...,love_im_dont
2,2,0,Female,\nLove me when I'm broke or when I'm filthy fu...,nigga_niggas_bitch
3,3,0,Male,"\n\n\nUgh, Killa!\nBaby!\nKanye, this that 197...",nigga_niggas_bitch
4,4,1,Female,\nThe conquering lion\nShall break every chain...,jesus_praise_lord


In [ ]:
aggregated_lyrics['topic'].value_counts()

-1      2
 355    2
 369    2
 368    2
 367    2
       ..
 173    2
 172    2
 171    2
 515    1
 491    1
Name: topic, Length: 543, dtype: int64

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def get_top_words(text, n=20):
    vec = CountVectorizer(stop_words='english').fit([text])
    bag_of_words = vec.transform([text])
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:n]
aggregated_lyrics['top_words'] = aggregated_lyrics['cleaned_lyrics'].apply(lambda x: get_top_words(x))

In [ ]:
aggregated_lyrics.head()

,Unnamed: 0,topic,gender,cleaned_lyrics,topic_label,top_words
0,0,-1,Female,"{entire song set to the melody of R. Kelly's ""...",love_im_dont,"[(don, 6171), (oh, 6098), (love, 5869), (know,..."
1,1,-1,Male,\n(*harmonizing*)\nAll my love is all I have\n...,love_im_dont,"[(love, 11719), (don, 11513), (like, 10311), (..."
2,2,0,Female,\nLove me when I'm broke or when I'm filthy fu...,nigga_niggas_bitch,"[(like, 8239), (got, 6193), (don, 5269), (bitc..."
3,3,0,Male,"\n\n\nUgh, Killa!\nBaby!\nKanye, this that 197...",nigga_niggas_bitch,"[(like, 193341), (nigga, 163882), (got, 143641..."
4,4,1,Female,\nThe conquering lion\nShall break every chain...,jesus_praise_lord,"[(lord, 5077), (oh, 4519), (god, 4142), (jesus..."


In [ ]:
!pip install numpy gensim

In [ ]:
!pip install matplotlib seaborn

Cosine Similarity: This function computes the cosine similarity between two word embeddings.

Mean Embedding Similarity: This function computes the average cosine similarity between each word in a target set and an attribute set.

Differential Association: This calculates the WEAT score, which is the differential association between two sets of target words and two sets of attribute words.

WEAT Effect Size: This calculates the effect size, a measure of how large the difference in associations is.

Target and Attribute Sets: You need to define these sets based on your specific analysis goal.

Target words are typically chosen to represent two different groups that you want to compare for bias. WEAT calculates how strongly each set of target words is associated with each set of attribute words. If one set of target words is more closely associated with positive attribute words (e.g., 'joy', 'peace', 'love') than the other, this might indicate a bias in the embedding space.Quantitative Analysis: The strength of these associations is quantified using cosine similarity in the embedding space. This provides a numerical measure of bias, which is the WEAT score.

In [ ]:
!wget -c "http://nlp.stanford.edu/data/glove.6B.zip"
!unzip glove.6B.zip
from gensim.scripts.glove2word2vec import glove2word2vec

glove_input_file = 'glove.6B.100d.txt'  # Adjust the file name as needed
word2vec_output_file = 'glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

--2023-12-25 17:48:46--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-12-25 17:48:47--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-12-25 17:48:47--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

<ipython-input-20-6ce0bcdfbacc>:7: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


(400000, 100)

if you want to display the topics as words instead of numeric labels in your WEAT analysis. you need to map the numeric topic labels to their corresponding words. You can achieve this by creating a mapping dictionary that associates each topic label with a list of words representing that topic.

In [ ]:
from gensim.models import KeyedVectors

word_embeddings_model = KeyedVectors.load_word2vec_format('glove.6B.100d.txt.word2vec', binary=False)

attribute sets

In [ ]:
male_words = ['he', 'him', 'his', 'father', 'papa', 'dad', 'son', 'uncle', 'grandfather', 'grandpa', 'man', 'male', 'brother', 'husband', 'boyfriend', 'sir', 'king', 'guy', 'father-in-law', 'son-in-law', 'nephew', 'boy']
female_words = ['she', 'her', 'hers', 'mother', 'mama', 'daughter', 'aunt', 'auntie', 'grandmother', 'woman', 'female', 'sister', 'mom', 'wife', 'girlfriend', 'madam', 'queen', 'gal', 'niece', 'grandmother-in-law', 'daughter-in-law', 'lady', 'miss', 'sis', 'girl']

#This variable is supposed to represent another set of attribute words for the WEAT analysis, serving as a basis for comparison against the attribute words extracted from each topic.
male_words = [word for word in male_words if word in word_embeddings_model.key_to_index]
female_words = [word for word in female_words if word in word_embeddings_model.key_to_index]

In [ ]:
df1 = df.drop_duplicates('topic')
topic_label_df = df1[['topic', 'topic_label']]
# topic_label_dictionary = topic_label_df.to_dict()

In [ ]:
topic_label_dictionary = topic_label_df.to_dict('records')

In [ ]:
topic_label_dictionary

[{'topic': 7, 'topic_label': 'mi_dem_yuh'},
 {'topic': 0, 'topic_label': 'nigga_niggas_bitch'},
 {'topic': 434, 'topic_label': 'bicyclette_ma bicyclette_bicyclette ma'},
 {'topic': 398, 'topic_label': 'dog status_movin rank_status im'},
 {'topic': 485, 'topic_label': 'sideline ho_woke girl_candy lies'},
 {'topic': 272, 'topic_label': 'im stickin_stickin im_stickin'},
 {'topic': 214, 'topic_label': 'rich ass_money money_th law'},
 {'topic': 526, 'topic_label': 'did brooklyn_brooklyn_brooklyn did'},
 {'topic': 50, 'topic_label': 'ghetto_ghetto ghetto_im ghetto'},
 {'topic': 94, 'topic_label': 'country_hank_thats right'},
 {'topic': 286, 'topic_label': 'dorothy_dorothy parker_parker dorothy'},
 {'topic': 98, 'topic_label': 'military time_military_time military'},
 {'topic': 261, 'topic_label': 'em youre_worth em_ahahah'},
 {'topic': 357, 'topic_label': 'wanna holler_holler_holler throw'},
 {'topic': 90, 'topic_label': 'pills_pills pills_drugs'},
 {'topic': 101, 'topic_label': 'tut_tut tut

In [ ]:
def get_topic_label(topic):
  for record in topic_label_dictionary:
    if topic == record['topic']:
      return record['topic_label']

  return 'Topic label not found!!'

In [ ]:
import numpy as np
from gensim.models import KeyedVectors
from nltk.corpus import wordnet as wn

import numpy as np
from nltk.corpus import wordnet as wn

# Ensure you have downloaded the necessary NLTK data
import nltk
nltk.download('wordnet')

def cosine_similarity(embedding1, embedding2):
    norm1 = np.linalg.norm(embedding1)
    norm2 = np.linalg.norm(embedding2)
    return np.dot(embedding1, embedding2) / (norm1 * norm2)

def get_concept_words(word):
    synsets = wn.synsets(word)
    return [lemma.name().replace('_', ' ') for synset in synsets for lemma in synset.lemmas() if synset.pos() in {'n', 'v'}]

def flatten(list_of_lists):
    return [item for sublist in list_of_lists for item in sublist]

def average_embedding(words, embeddings_model):
    valid_embeddings = [embeddings_model[word] for word in words if word in embeddings_model.key_to_index]
    return np.mean(valid_embeddings, axis=0) if valid_embeddings else np.zeros(embeddings_model.vector_size)

# SC-WEAT effect size calculation function with concept words for the target set
def sc_weat_effect_size_target_concepts(target_words, male_words, female_words, embeddings_model, average_score = True):
    # Generate concept words for the target set
    target_concepts = flatten([get_concept_words(word) for word in target_words])
    word_scweat_dict = []

    # Calculate average embeddings for male and female attribute sets
    male_avg_embedding = average_embedding(male_words, embeddings_model)
    female_avg_embedding = average_embedding(female_words, embeddings_model)

    diffs = []
    for target_concept in target_concepts:
        if target_concept in embeddings_model.key_to_index:
            target_embedding = embeddings_model[target_concept]
            diff = cosine_similarity(target_embedding, male_avg_embedding) - \
                   cosine_similarity(target_embedding, female_avg_embedding)
            diffs.append(diff)

            word_scweat_dict.append({
                'concept_word': target_concept,
                'scweat_score': diff
            })

    if average_score:
      effect_size = np.mean(diffs) / np.std(diffs) if diffs else 0
      return effect_size

    else:
      return word_scweat_dict

# A positive effect size suggests that the concept words associated with a topic are more closely
# related to the male words than to the female words. This could indicate a male-gendered bias in the context of that topic.


def print_sc_weat_results(topic, score):
    print(f"Topic: {topic}")
    print(f"  SCWEAT effect: {score}")
    print("")

# Define your target and attribute word sets
male_words = [word for word in male_words if word in word_embeddings_model.key_to_index]
female_words = [word for word in female_words if word in word_embeddings_model.key_to_index]

# Create a mapping dictionary to associate topic labels with words
topic_words_mapping = {}
sc_weat_results = []

for topic_label in aggregated_lyrics['topic'].unique():
    # Logic to obtain top words for each topic
    top_words = aggregated_lyrics[(aggregated_lyrics['topic'] == topic_label)]['top_words'].tolist()
    top_words = flatten(top_words)
    top_words_cleared = [item[0] for item in top_words]
    topic_words_mapping[topic_label] = top_words_cleared
    target_set=top_words_cleared
    topic = get_topic_label(topic_label)

    if target_set:
        effect_size = sc_weat_effect_size_target_concepts(target_set, male_words, female_words, word_embeddings_model)
        print_sc_weat_results(topic,  effect_size)
        sc_weat_results.append({
            "topic_number": str(topic_label),
            "topic_label": topic,
            "effect_size": effect_size,
        })
    else:
        print(f"Topic {topic} - Not enough data for SC-WEAT analysis")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Topic: love_im_dont
  SCWEAT effect: 0.7588549852371216

Topic: nigga_niggas_bitch
  SCWEAT effect: 0.5312932729721069

Topic: jesus_praise_lord
  SCWEAT effect: 0.35344961285591125

Topic: body_girl_baby
  SCWEAT effect: 0.6045235395431519

Topic: dance_funky_dance dance
  SCWEAT effect: 0.6529636979103088

Topic: ayy ayy_change_long sentiment
  SCWEAT effect: 0.899697482585907

Topic: christmas_santa_year
  SCWEAT effect: 0.5330460667610168

Topic: wi_ye_sae
  SCWEAT effect: 0.4073067009449005

Topic: mi_dem_yuh
  SCWEAT effect: 0.8305016160011292

Topic: shes_shell_borderlines
  SCWEAT effect: 0.604462206363678

Topic: money_money money_dub dub
  SCWEAT effect: 0.9158362150192261

Topic: love_make love_baby
  SCWEAT effect: 0.6283813118934631

Topic: tears_heart_wish
  SCWEAT effect: 0.5240224003791809

Topic: blues_worried_blues blues
  SCWEAT effect: 0.5624501705169678

Topic: id_world world_wouldnt
  SCWEAT effect: 0.6932216286659241

Topic: shes_shes got_cameron
  SCWEAT effect:

In [ ]:
sc_weat_results

[{'topic_number': '-1',
  'topic_label': 'love_im_dont',
  'effect_size': 0.758855},
 {'topic_number': '0',
  'topic_label': 'nigga_niggas_bitch',
  'effect_size': 0.5312933},
 {'topic_number': '1',
  'topic_label': 'jesus_praise_lord',
  'effect_size': 0.3534496},
 {'topic_number': '2',
  'topic_label': 'body_girl_baby',
  'effect_size': 0.60452354},
 {'topic_number': '3',
  'topic_label': 'dance_funky_dance dance',
  'effect_size': 0.6529637},
 {'topic_number': '4',
  'topic_label': 'ayy ayy_change_long sentiment',
  'effect_size': 0.8996975},
 {'topic_number': '5',
  'topic_label': 'christmas_santa_year',
  'effect_size': 0.53304607},
 {'topic_number': '6', 'topic_label': 'wi_ye_sae', 'effect_size': 0.4073067},
 {'topic_number': '7', 'topic_label': 'mi_dem_yuh', 'effect_size': 0.8305016},
 {'topic_number': '8',
  'topic_label': 'shes_shell_borderlines',
  'effect_size': 0.6044622},
 {'topic_number': '9',
  'topic_label': 'money_money money_dub dub',
  'effect_size': 0.9158362},
 {'t

In [ ]:
scweat = pd.DataFrame(sc_weat_results)

In [ ]:
scweat.head()

,topic_number,topic_label,effect_size
0,-1,love_im_dont,0.758855
1,0,nigga_niggas_bitch,0.531293
2,1,jesus_praise_lord,0.353450
3,2,body_girl_baby,0.604524
4,3,dance_funky_dance dance,0.652964


In [ ]:
# Create a mapping dictionary to associate topic labels with words
topic_words_mapping = {}
sc_weat_results_word = []

for topic_label in aggregated_lyrics['topic'].unique():
    # Logic to obtain top words for each topic
    top_words = aggregated_lyrics[(aggregated_lyrics['topic'] == topic_label)]['top_words'].tolist()
    top_words = flatten(top_words)
    top_words_cleared = [item[0] for item in top_words]
    topic_words_mapping[topic_label] = top_words_cleared
    target_set=top_words_cleared
    topic = get_topic_label(topic_label)

    if target_set:
        word_scores = sc_weat_effect_size_target_concepts(target_set, male_words, female_words, word_embeddings_model, average_score = False)
        # print_sc_weat_results(topic,  effect_size)
        sc_weat_results_word.append({
            "topic_number": str(topic_label),
            "topic_label": topic,
            "word_scores": word_scores,
        })
    else:
        print(f"Topic {topic} - Not enough data for SC-WEAT analysis")

In [ ]:
df_word_scweat = pd.DataFrame(sc_weat_results_word)

In [ ]:
df_word_scweat.head()

,topic_number,topic_label,word_scores
0,-1,love_im_dont,"[{'concept_word': 'preceptor', 'scweat_score':..."
1,0,nigga_niggas_bitch,"[{'concept_word': 'like', 'scweat_score': 0.06..."
2,1,jesus_praise_lord,"[{'concept_word': 'overlord', 'scweat_score': ..."
3,2,body_girl_baby,"[{'concept_word': 'like', 'scweat_score': 0.06..."
4,3,dance_funky_dance dance,"[{'concept_word': 'dance', 'scweat_score': -0...."


In [ ]:
scweat['word_scores'] = df_word_scweat['word_scores']

In [ ]:
scweat.head(10)

,topic_number,topic_label,effect_size,word_scores
0,-1,love_im_dont,0.758855,"[{'concept_word': 'preceptor', 'scweat_score':..."
1,0,nigga_niggas_bitch,0.531293,"[{'concept_word': 'like', 'scweat_score': 0.06..."
2,1,jesus_praise_lord,0.353450,"[{'concept_word': 'overlord', 'scweat_score': ..."
3,2,body_girl_baby,0.604524,"[{'concept_word': 'like', 'scweat_score': 0.06..."
4,3,dance_funky_dance dance,0.652964,"[{'concept_word': 'dance', 'scweat_score': -0...."
5,4,ayy ayy_change_long sentiment,0.899697,"[{'concept_word': 'preceptor', 'scweat_score':..."
6,5,christmas_santa_year,0.533046,"[{'concept_word': 'baby', 'scweat_score': -0.1..."
7,6,wi_ye_sae,0.407307,"[{'concept_word': 'semen', 'scweat_score': -0...."
8,7,mi_dem_yuh,0.830502,"[{'concept_word': 'like', 'scweat_score': 0.06..."
9,8,shes_shell_borderlines,0.604462,"[{'concept_word': 'love', 'scweat_score': -0.0..."


In [ ]:
scweat.to_csv(bertopic_folder + 'scweat_results.csv')